In [3]:
from collections import Counter
import pandas as pd
from math import log10

training_set = pd.read_csv('covid_training.tsv', delimiter='\t')
test_set = pd.read_csv('covid_test_public.tsv', delimiter='\t')


In [4]:
ov = Counter()
ov_no = Counter()
ov_yes = Counter()
fv_no = Counter()
fv_yes = Counter()

training_set['text'].str.lower().str.split().apply(ov.update)
training_set[training_set['q1_label'] == 'no']['text'].str.lower().str.split().apply(ov_no.update)
training_set[training_set['q1_label'] == 'yes']['text'].str.lower().str.split().apply(ov_yes.update)

ov_no = pd.Series(Counter({k:ov_no[k] for k, v in ov.items()}))
ov_yes = pd.Series(Counter({k:ov_yes[k] for k, v in ov.items()}))

training_set[training_set['q1_label'] == 'no']['text'].str.lower().str.split().apply(fv_no.update)
training_set[training_set['q1_label'] == 'yes']['text'].str.lower().str.split().apply(fv_yes.update)

fv = Counter({k:v for k, v in ov.items() if v > 1})
fv_no = pd.Series(Counter({k:fv_no[k] for k, v in fv.items()}))
fv_yes = pd.Series(Counter({k:fv_yes[k] for k, v in fv.items()}))


In [5]:
alpha = 0.01

logs_ov = pd.DataFrame(columns = ov.keys())
logs_ov = logs_ov.append(ov_no.apply(lambda x : log10((x + alpha)/(ov_no.sum()+ (alpha * len(ov))))), ignore_index=True)
logs_ov = logs_ov.append(ov_yes.apply(lambda x : log10((x + alpha)/(ov_yes.sum() + (alpha * len(ov))))), ignore_index=True)

logs_fv = pd.DataFrame(columns = fv.keys())
logs_fv = logs_fv.append(fv_no.apply(lambda x : log10((x + alpha)/(fv_no.sum() + (alpha * len(fv))))), ignore_index=True)
logs_fv = logs_fv.append(fv_yes.apply(lambda x : log10((x + alpha)/(fv_yes.sum() + (alpha * len(fv))))), ignore_index=True)    
    

In [6]:
logs_ov

,for,the,average,american,best,way,to,tell,if,you,...,https://t.co/inykd5tctl,50%,rate.,"infected,",die.,obviously,ebolaâbut,34x,https://t.co/bcyoeedray,https://t.co/ga2rdghpiq
0,-2.046349,-1.457349,-3.573634,-2.974811,-2.974811,-2.878118,-1.609426,-3.274760,-2.347251,-1.905765,...,-3.573634,-5.577956,-5.577956,-5.577956,-5.577956,-5.577956,-5.577956,-5.577956,-5.577956,-5.577956
1,-1.994466,-1.442239,-5.948757,-3.103039,-3.645561,-3.169883,-1.564924,-3.470191,-2.550643,-2.241102,...,-5.948757,-3.944436,-3.645561,-3.944436,-3.944436,-3.944436,-3.944436,-3.944436,-3.944436,-3.944436


In [7]:
logs_fv

,for,the,american,best,way,to,tell,if,you,have,...,hotel,focusing,sars-cov-2,youâd,â£574.70,ng,na,60s.,chloroquine,rate.
0,-1.922465,-1.333465,-2.850927,-2.850927,-2.754234,-1.485542,-3.150876,-2.223367,-1.781881,-2.038931,...,-3.449750,-5.454072,-5.454072,-5.454072,-5.454072,-5.454072,-5.454072,-5.454072,-5.454072,-5.454072
1,-1.868722,-1.316494,-2.977295,-3.519817,-3.044138,-1.439180,-3.344446,-2.424899,-2.115358,-2.231837,...,-3.818692,-3.519817,-3.519817,-3.519817,-3.519817,-3.519817,-3.519817,-3.519817,-3.519817,-3.519817


In [10]:
fv_accuracy = 0
fv_yes_tp = 0;
fv_yes_fp = 0;
fv_yes_tn = 0;
fv_yes_fn = 0;
fv_no_tp = 0;
fv_no_fp = 0;
fv_no_tn = 0;
fv_no_fn = 0;

ov_accuracy = 0
ov_yes_tp = 0;
ov_yes_fp = 0;
ov_yes_tn = 0;
ov_yes_fn = 0;
ov_no_tp = 0;
ov_no_fp = 0;
ov_no_tn = 0;
ov_no_fn = 0;

test_set.columns = training_set.columns

f = open('trace_NB-BOW-OV.txt', 'w')

for index, values in test_set.iterrows():
    score_yes = log10(len(training_set[training_set['q1_label'] == 'yes']) / len(training_set))
    score_no = log10(len(training_set[training_set['q1_label'] == 'no']) / len(training_set))
    for word in set(values['text'].lower().split(' ')):
        if word in logs_ov.columns:
            score_yes += logs_ov[word][1]
            score_no += logs_ov[word][0]
    if score_yes >= score_no:
        c = 'yes'
    else:
        c = 'no'
    real_c = values['q1_label']

    if c == real_c : 
        ov_accuracy += 1
        result = '  correct' 
        if c == 'no':
            ov_no_tp +=1
            ov_yes_tn +=1
        else :
            ov_yes_tp +=1
            ov_no_tn +=1
    else: 
        result = '  wrong'
        if c == 'no':
            ov_no_fp +=1
            ov_yes_fn +=1
        else :
            ov_yes_fp +=1
            ov_no_fn +=1
    f.write(str(values['tweet_id']) + '  ' + c + f"  {max(score_yes, score_no):E}  " + real_c + result + '\n')
    
f.close()

ov_accuracy = ov_accuracy / len(test_set)
ov_yes_precision = ov_yes_tp / (ov_yes_tp + ov_yes_fp)
ov_no_precision = ov_no_tp / (ov_no_tp + ov_no_fp)
ov_yes_recall = ov_yes_tp / (ov_yes_tp + ov_yes_fn)
ov_no_recall = ov_no_tp / (ov_no_tp + ov_no_fn)
ov_yes_f1 = ov_yes_tp / (ov_yes_tp + 0.5 * (ov_yes_fp+ ov_yes_fn) )
ov_no_f1 = ov_no_tp / (ov_no_tp + 0.5 * (ov_no_fp+ ov_no_fn) )

f = open('eval_NB-BOW-OV.txt', 'w')
f.write("%f \n%f  %f\n%f  %f\n%f  %f\n" % (ov_accuracy, ov_yes_precision, ov_no_precision, ov_yes_recall, ov_no_recall, ov_yes_f1, ov_no_f1))
f.close()

f = open('trace_NB-BOW-FV.txt', 'w')

for index, values in test_set.iterrows():
    score_yes = log10(len(training_set[training_set['q1_label'] == 'yes']) / len(training_set))
    score_no = log10(len(training_set[training_set['q1_label'] == 'no']) / len(training_set))
    for word in set(values['text'].lower().split(' ')):
        if word in logs_fv.columns:
            score_yes += logs_fv[word][1]
            score_no += logs_fv[word][0]
    if score_yes >= score_no:
        c = 'yes'
    else:
        c = 'no'
        
    real_c = values['q1_label']

    if c == real_c : 
        fv_accuracy += 1
        result = '  correct' 
        if c == 'no':
            fv_no_tp +=1
            fv_yes_tn +=1
        else :
            fv_yes_tp +=1
            fv_no_tn +=1
    else: 
        result = '  wrong'
        if c == 'no':
            fv_no_fp +=1
            fv_yes_fn +=1
        else :
            fv_yes_fp +=1
            fv_no_fn +=1
    f.write(str(values['tweet_id']) + '  ' + c + f"  {max(score_yes, score_no):E}  " + real_c + result + '\n')
    
f.close()

fv_accuracy = fv_accuracy / len(test_set)
fv_yes_precision = fv_yes_tp / (fv_yes_tp + fv_yes_fp)
fv_no_precision = fv_no_tp / (fv_no_tp + fv_no_fp)
fv_yes_recall = fv_yes_tp / (fv_yes_tp + fv_yes_fn)
fv_no_recall = fv_no_tp / (fv_no_tp + fv_no_fn)
fv_yes_f1 = fv_yes_tp / (fv_yes_tp + 0.5 * (fv_yes_fp+ fv_yes_fn) )
fv_no_f1 = fv_no_tp / (fv_no_tp + 0.5 * (fv_no_fp+ fv_no_fn) )

f = open('eval_NB-BOW-FV.txt', 'w')
f.write("%f \n%f  %f\n%f  %f\n%f  %f\n" % (fv_accuracy, fv_yes_precision, fv_no_precision, fv_yes_recall, fv_no_recall, fv_yes_f1, fv_no_f1))
f.close()

In [11]:
from platform import python_version

print(python_version())

3.8.5
